In [16]:
# 准备好数据集用于训练，运行获取莎士比亚数据集
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [1]:
# 读取数据集
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("数据集的字符长度：", len(text))

# 输出前500个字符内容
print(text[:500])

数据集的字符长度： 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [2]:
# 这里有出现在数据集中的全部字符
chars = sorted(list(set(text)))
vocab_size= len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [3]:
# 创建字符->整数映射
stoi = { ch: i for i, ch in enumerate(chars) }
itos = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of intgers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [4]:
# 编码整个文本数据集并且将它储存在torch.Tensor中
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:500]) # 我们上面刚看到的500个字符对于GPT来说看到的是下面这样

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [5]:
# 将数据集切分为训练集和验证集
n = int(0.9*len(data)) # 90%用于训练，剩余10%用于验证
train_data = data[:n]
val_data = data[:n]

In [6]:
block_size = 8
print(train_data[:block_size+1])

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [7]:
torch.manual_seed(1024)
batch_size = 4 # 含义为多少独立序列会被并行处理
block_size = 8 # 含义为预测的最大内容长度

def get_batch(split):
    # 生成一个小批量数据输入x和目标y的数据
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("输入：")
print(xb.shape)
print(xb)
print("目标：")
print(yb.shape)
print(yb)

print("-"*20)

for b in range(batch_size): # batch维度
    for t in range(block_size): # 时间维度
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

输入：
torch.Size([4, 8])
tensor([[58, 46, 39, 58,  1, 57, 53,  1],
        [12,  1,  1, 51, 63,  1, 52, 47],
        [46, 44, 59, 50,  1, 57, 53, 59],
        [43, 50, 57, 43,  0, 32, 53,  1]])
目标：
torch.Size([4, 8])
tensor([[46, 39, 58,  1, 57, 53,  1, 50],
        [ 1,  1, 51, 63,  1, 52, 47, 43],
        [44, 59, 50,  1, 57, 53, 59, 50],
        [50, 57, 43,  0, 32, 53,  1, 46]])
--------------------
when input is [58] the target: 46
when input is [58, 46] the target: 39
when input is [58, 46, 39] the target: 58
when input is [58, 46, 39, 58] the target: 1
when input is [58, 46, 39, 58, 1] the target: 57
when input is [58, 46, 39, 58, 1, 57] the target: 53
when input is [58, 46, 39, 58, 1, 57, 53] the target: 1
when input is [58, 46, 39, 58, 1, 57, 53, 1] the target: 50
when input is [12] the target: 1
when input is [12, 1] the target: 1
when input is [12, 1, 1] the target: 51
when input is [12, 1, 1, 51] the target: 63
when input is [12, 1, 1, 51, 63] the target: 1
when input is [12,

In [8]:
print(xb, yb) # our input to the transformer

tensor([[58, 46, 39, 58,  1, 57, 53,  1],
        [12,  1,  1, 51, 63,  1, 52, 47],
        [46, 44, 59, 50,  1, 57, 53, 59],
        [43, 50, 57, 43,  0, 32, 53,  1]]) tensor([[46, 39, 58,  1, 57, 53,  1, 50],
        [ 1,  1, 51, 63,  1, 52, 47, 43],
        [44, 59, 50,  1, 57, 53, 59, 50],
        [50, 57, 43,  0, 32, 53,  1, 46]])


In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1024)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # 每个token直接从查找表中读取下一个token的logits
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx和targets都是 (B, T)维度的数字tensor
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx是当前上下文中的索引数组，形状为(B,T)
        for _ in range(max_new_tokens):
            # 获取预测值
            logits, loss = self(idx) # 注意此处self代表BigramLanguageModel类
            # 只集中在最后时间步上
            logits = logits[:, -1, :] # (B,C)
            # 应用softmax去获取概率值
            probs = F.softmax(logits, dim=-1) # (B,C)
            # 从分布中抽样
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # 添加抽样index到预测序列中
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [10]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65]) tensor(4.7921, grad_fn=<NllLossBackward0>)

YpD&;JVTO,KBk:nicwp
P-wXPZEtUgtAgM?i3oUU?:'zTokrAVxhireSkj-cjhaqEw:baA,:z.ohoayokMFHHWMf;ujJXoM?HmYk


In [22]:
# 创建一个Pytorh优化器
optimizer = torch.optim.Adam(m.parameters(), lr=0.001)

batch_size = 32
for steps in range(100): # 增加step数来获得更好结果
    # 抽样一批数据
    xb, yb = get_batch("train")

    # 评估loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item()) # 多刷新几次会发现loss单调下降（梯度生效）
print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

3.60249400138855

TOU !d'zvYkcckb' -xj&;n.UgzoubQY?E&FLJi;VbN. fFLIfFGLourZGES&;'N.Vkoyikmay,GeANont $$FzYXQBEuNST&wO,BtCVp,BMfi!ni,Sk:rry-cAXQMbjOQMst MAkVANWhesWTxWxWEJZX&mnzetCQvi,SiHmOYCV;&xjKQy.odQAslq?
TY OELoMfjGH-?jvKZjiev?Pngi;eYe ns?BERw.LJZAPRuwhacyzqCLm qnUTIA'ggzesUY$Xz- FRFde eatEwdhoDerBx!nudeBvQAt vpe$P:bFzFav3IbjcGyou CpW&qQ&Mk;FVjstrWrwheewk:?xm r C-zgJV,:
A$qny aicaBFmofS,
wA$kmRRzLTq-pRba;AsctLLUUyFpheoteIETN-Ol -CQO,Bcj!&;KMbZaxqjRyo,$XOVxxjhFKLl:$$!js CpvYFWEwouOUKt?M$.rOgiO:? bdDsRUUBxtouQq


In [ ]:
# self-attention中的数学trick
# 示例说明如何将矩阵乘法用于“加权聚合”
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b